Below is a Jupyter Notebook-style tutorial that walks through the process of fetching seismic event data from the ISC catalog using the obspy library, and saving the data to a CSV file.

In [4]:
# seismic_event_extraction.ipynb

# Seismic Event Extraction from ISC Catalog using ObsPy
# =====================================

# **Objective**: This notebook demonstrates how to use the `obspy` library to fetch seismic event data 
# from the ISC catalog for a specific geographic region and time range. The extracted data will include 
# the origin time, latitude, longitude, magnitude, and depth of each event. Finally, the data will be 
# saved to a CSV file.

# Importing Necessary Libraries
# -----------------------------

# We'll begin by importing the necessary libraries. The `obspy` library will be used to interact with the 
# seismic data, and `pandas` will be used for data manipulation and saving to a CSV file.
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import pandas as pd

Setting Up the FDSN Client
--------------------------
The FDSN client allows us to access seismic data from various data centers. In this tutorial, we will use
the IRIS client to fetch events from the ISC catalog.

In [5]:
# Initialize the IRIS client
client = Client("IRIS")

Defining the Search Parameters
------------------------------
We'll define the geographic region, time range, and magnitude range for the events we want to extract.
The region is defined by the latitude and longitude boundaries, and the time range is specified using
the UTCDateTime format.

In [6]:
# Define the range of catalog
minlat = 29
maxlat = 32
minlon = 77
maxlon = 81
starttime = UTCDateTime("2010-01-01")
endtime = UTCDateTime("2020-01-01")
minmag = 0
maxmag = 10

Fetching Events from the ISC Catalog
------------------------------------
Using the defined parameters, we'll fetch the seismic event data from the ISC catalog. The data includes
events that occurred within the specified region and time range.

In [7]:
# Fetch events from the ISC catalog
cat = client.get_events(starttime=starttime, endtime=endtime,
                        minlatitude=minlat, maxlatitude=maxlat,
                        minlongitude=minlon, maxlongitude=maxlon,
                        minmagnitude=minmag, maxmagnitude=maxmag,
                        mindepth=0, maxdepth=100,
                        catalog="ISC")

Extracting Event Details
------------------------
Now, we'll loop through the fetched events to extract relevant information such as the origin time,
latitude, longitude, magnitude, and depth. This data will be stored in a list.

In [8]:
# Prepare a list to store event details
event_data = []

# Loop through the events and extract relevant information
for event in cat:
    origin = event.preferred_origin() or event.origins[0]
    magnitude = event.preferred_magnitude() or event.magnitudes[0]

    # Extract latitude, longitude, magnitude, depth, and origin time
    lat = origin.latitude
    lon = origin.longitude
    mag = magnitude.mag
    depth = origin.depth / 1000 if origin.depth is not None else None  # Convert depth to kilometers
    origin_time = origin.time  # UTCDateTime

    # Append the event data (origin_time, lat, lon, mag, depth) to the list
    event_data.append([origin_time, lat, lon, mag, depth])


Creating a DataFrame and Saving to CSV
--------------------------------------
The extracted event data is then converted into a Pandas DataFrame, which makes it easier to manipulate
and save to a file. We'll save the DataFrame to a CSV file for further analysis.

In [9]:
# Convert the list to a Pandas DataFrame
df = pd.DataFrame(event_data, columns=["Origin Time", "Latitude", "Longitude", "Magnitude", "Depth"])

# Save the DataFrame to a CSV file
df.to_csv("./seismic_events.csv", index=False)

Viewing the Extracted Data
--------------------------
Finally, let's take a quick look at the first few rows of the extracted data to ensure everything was
captured correctly.

In [ ]:
# Print the DataFrame
print(df.head())